In [54]:
from unidecode import unidecode
import random
import requests

In [55]:
class JogoDeForca:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).split('\n')
            self.content = [unidecode(p.lower()).replace('ç', 'c') for p in self.content]
        else:
            print("Erro: ", r.status_code)
    
    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        return len(self.palavra)

In [56]:
def chuta_letra(lista_palavras, letras_restantes):
    quantidade_letras = {}
    for palavra in lista_palavras:
        for letra in letras_restantes:
            if letra in palavra:
                if letra in quantidade_letras:
                    quantidade_letras[f'{letra}'] += 1
                else:
                    quantidade_letras[f'{letra}'] = 1
    mais_recorrente = max(quantidade_letras.items(), key=lambda x: x[1])[0]
    return mais_recorrente
    

In [57]:
def remove_com_letra(letra, lista_palavras):
    sem_letra = []
    for palavra in lista_palavras:
        if letra not in palavra:
            sem_letra.append(palavra)
    return sem_letra

In [58]:
def remove_sem_letra(resposta, letra, lista_palavras):
    com_letra = []

    indice = resposta.find(letra)
    ocorrencias = []
    while indice != -1:
        ocorrencias.append(indice)
        indice = resposta.find(letra, indice + 1)

    
    for palavra in lista_palavras:
        corresponde = 0
        for indice in ocorrencias:
            if palavra[indice] == letra:
                corresponde += 1
        if corresponde == len(ocorrencias):
            com_letra.append(palavra)
            
                
    return com_letra

In [59]:
def remove_por_tamanho(tamanho, lista_palavras):
    lista_mesmos_tamanhos = []
    for palavra in lista_palavras:
        if len(palavra) == tamanho:
            lista_mesmos_tamanhos.append(palavra)
    return lista_mesmos_tamanhos

In [60]:
def calcula_certas(resposta, letra):
    certas = 0
    for l in resposta:
        if l == letra:
            certas += 1
    return certas

In [61]:
def jogador(content, palavra, vidas):
    alfabeto = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    content = remove_por_tamanho(len(palavra), content)
    letra_q_ja_foram = []
    qtd_certas = 0
    while vidas > 0:
        letra = chuta_letra(content, alfabeto)
        letra_q_ja_foram.append(letra)
        if letra in palavra:
            content = remove_sem_letra(palavra, letra, content)
            qtd_certas += calcula_certas(palavra, letra)
        else:
            content = remove_com_letra(letra, content)
            vidas -= 1
        alfabeto.remove(letra)
        if qtd_certas == len(palavra):
            return content[0]
    return content[0]

In [63]:
certas = 0
erradas = 0
for _ in range(50):
    jogo = JogoDeForca()
    jogo.novo_jogo()
    print(f"Palavra certa: {jogo.palavra}")
    chute = jogador(jogo.content, jogo.palavra, jogo.vidas)
    print(f"Palavra chutada: {chute}")
    if jogo.palavra == chute:
        certas += 1
    else:
        erradas +=1 
print(f"Certas: {certas}\nErradas: {erradas}")


Palavra certa: apegassemos
Palavra chutada: apegassemos
Palavra certa: peninsulas
Palavra chutada: peninsulas
Palavra certa: apimentem
Palavra chutada: apimentem
Palavra certa: escandalosos
Palavra chutada: escandalosos
Palavra certa: lupa
Palavra chutada: lupa
Palavra certa: sucumbisseis
Palavra chutada: sucumbisseis
Palavra certa: devassador
Palavra chutada: devassador
Palavra certa: insidiam
Palavra chutada: insidiam
Palavra certa: cutelarias
Palavra chutada: cutelarias
Palavra certa: requerermos
Palavra chutada: requerermos
Palavra certa: envernizava
Palavra chutada: envernizava
Palavra certa: seviciarieis
Palavra chutada: seviciarieis
Palavra certa: divagante
Palavra chutada: divagante
Palavra certa: fracionara
Palavra chutada: fracionara
Palavra certa: adulteremo
Palavra chutada: adulteremo
Palavra certa: diminuira
Palavra chutada: diminuira
Palavra certa: arborize
Palavra chutada: arborize
Palavra certa: mimeografo
Palavra chutada: mimeografo
Palavra certa: incumbo
Palavra chuta